# Decomposition algorithm

In [1]:
using DynamicPolynomials, MultivariateSeries, LinearAlgebra
X = @polyvar x1 x2 x3

(x1, x2, x3)

We want to find a sparse representation of the following series known up to degree 3:

In [2]:
sigma = dual(6.0 + 4.0*x1 + 15.0*x2 + 6.0*x3 + 6.0*x1^2 + 20.0*x1*x2 + 4.0*x1*x3 + 43.0*x2^2 + 15.0*x2*x3 + 6.0*x3^2 - 26.0*x1^3 + 30.0*x1^2*x2 + 6.0*x1^2*x3 + 72.0*x1*x2^2 + 20.0*x1*x2*x3 + 4.0*x1*x3^2 + 129.0*x2^3 + 43.0*x2^2*x3 + 15.0*x2*x3^2 + 6.0*x3^3)

6.0 + 6.0dx3 + 15.0dx2 + 4.0dx1 + 6.0dx3^2 + 15.0dx2*dx3 + 43.0dx2^2 + 4.0dx1*dx3 + 20.0dx1*dx2 + 6.0dx1^2 + 6.0dx3^3 + 15.0dx2*dx3^2 + 43.0dx2^2dx3 + 129.0dx2^3 + 4.0dx1*dx3^2 + 20.0dx1*dx2*dx3 + 72.0dx1*dx2^2 + 6.0dx1^2dx3 + 30.0dx1^2dx2 - 26.0dx1^3

In [3]:
L1 = monomials(X,0:1)
L2 = monomials(X,0:2)

10-element MonomialVector{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, Graded{LexOrder}}:
 1
 x3
 x2
 x1
 x3²
 x2x3
 x2²
 x1x3
 x1x2
 x1²

In [4]:
H = hankel(sigma,L1,L2)

4×10 Matrix{Float64}:
  6.0   6.0  15.0   4.0   6.0  15.0   43.0   4.0  20.0    6.0
  6.0   6.0  15.0   4.0   6.0  15.0   43.0   4.0  20.0    6.0
 15.0  15.0  43.0  20.0  15.0  43.0  129.0  20.0  72.0   30.0
  4.0   4.0  20.0   6.0   4.0  20.0   72.0   6.0  30.0  -26.0

The rank of $H_{\sigma}$ will give us an idea on the dimension of $\mathcal{A}_\sigma$.

In [5]:
rank(H)

3

We check that $\{1, x_1, x_2\}$ is a basis of $\mathcal{A}_\sigma$: 

In [6]:
B0 = L1[1:3]

3-element MonomialVector{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, Graded{LexOrder}}:
 1
 x3
 x2

In [7]:
H0 = hankel(sigma, B0, B0)

3×3 Matrix{Float64}:
  6.0   6.0  15.0
  6.0   6.0  15.0
 15.0  15.0  43.0

In [8]:
rank(H0)

2

Let us compute the shifted (truncated) Hankel operators.

In [9]:
H1 = hankel(sigma, B0, B0*x1)
H2 = hankel(sigma, B0, B0*x2)
H3 = hankel(sigma, B0, B0*x3);
H  = [H1,H2,H3]
H[1]

3×3 Matrix{Float64}:
  4.0   4.0  20.0
  4.0   4.0  20.0
 20.0  20.0  72.0

In [10]:
M = [ H0^(-1)*H[i] for i in 1:3 ]
M[1]

LoadError: SingularException(2)

The eigenvalues and eigenvectors of $M_{x_1}$ are

We deduce the operators of multiplication by the variables in the basis $B_0$:

In [11]:
v, E = eigen(M[1])

LoadError: UndefVarError: `M` not defined

The matrices $M_{x_i}$ are diagonal in this basis:

In [12]:
D = [E^(-1)*M[i]*E for i in 1:3]
D[1]

LoadError: UndefVarError: `E` not defined

In [13]:
D[2]

LoadError: UndefVarError: `D` not defined

In [14]:
D[3]

LoadError: UndefVarError: `D` not defined

Looking at the corresponding terms on the diagonal, we get the coordinates of the points $\Xi$:

In [15]:
Xi = [ D[i][j,j] for i in 1:3, j in 1:3]

LoadError: UndefVarError: `D` not defined

We normalize the eigenvectors by $v_i \over v_i(\xi_i)$ and get the interpolation polynomials at the points $\xi_i$:

In [16]:
Dg = E'*vcat(fill(1.,1,3), Xi[1:2,:])
E = E*Dg^(-1)
U = E'*B0

LoadError: UndefVarError: `E` not defined

We deduce the weights $w_i=\sigma(u_i)$:

In [17]:
w = hankel(sigma, U, [L1[1]])

LoadError: UndefVarError: `U` not defined

Using the command `decompose`, we can get directly the same decomposition: 

In [18]:
w, Xi = decompose(sigma)

([2.0, 5.000000000000065, -1.0000000000000482], [-1.0 1.9999999999999987 4.000000000000004; 1.0 2.9999999999999982 2.0000000000000013; 1.0 1.0 1.0])

In [19]:
Xi

3×3 Matrix{Float64}:
 -1.0  2.0  4.0
  1.0  3.0  2.0
  1.0  1.0  1.0

In [20]:
w

3-element Vector{Float64}:
  2.0
  5.000000000000065
 -1.0000000000000482

The series decomposes as $2 \mathfrak{e}_{(-1,1,1)} + 5 \mathfrak{e}_{(2,3,1)} - \mathfrak{e}_{(4,2,1)}$.